In [19]:
import numpy as np
import pandas as pd
import math
from sympy import symbols, expand, N

# ---- HÀM ĐỌC DỮ LIỆU (Giữ nguyên) ----
def read_data_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        if 'x' not in df.columns or 'y' not in df.columns: return None, None
        return df['x'].to_numpy(), df['y'].to_numpy()
    except Exception as e:
        print(f"Lỗi: {e}")
        return None, None

# ---- CÁC HÀM CỐT LÕI (Giữ nguyên) ----
def checkEquidistant(x):
    if len(x) < 2: return None
    h = x[1] - x[0]
    if h <= 0 or not np.allclose(np.diff(x), h): return None
    return h

def factorial(k):
    return math.factorial(k)

def nhandathuc(b, c):
    n_b = len(b)
    a = np.zeros(n_b + 1)
    a[0] = b[0]
    for i in range(1, n_b): a[i] = b[i] - b[i-1] * c
    a[n_b] = -b[n_b-1] * c
    return a

def expand_poly_from_integer_roots(n):
    W = np.array([1.0])
    for root in range(n + 1): W = nhandathuc(W, root)
    return W

def poly_divide_by_monomial(poly, root):
    n = len(poly) - 1
    if n < 1: return np.array([0.0])
    quotient = np.zeros(n)
    quotient[0] = poly[0]
    for i in range(1, n): quotient[i] = poly[i] + quotient[i-1] * root
    return quotient

# ---- HÀM HỖ TRỢ IN ẤN (Giữ nguyên) ----
def format_poly_final(coeffs, var='x', precision=5):
    coeffs = np.round(coeffs, precision)
    poly_str = ""
    n = len(coeffs) - 1
    for i in range(len(coeffs)):
        coeff = coeffs[i]; power = n - i
        if abs(coeff) < 1e-12: continue
        sign = " - " if coeff < 0 else " + "
        if poly_str == "": sign = "-" if sign == " - " else ""
        coeff = abs(coeff)
        if power == 0: term = f"{coeff}"
        elif power == 1: term = f"{coeff}*{var}"
        else: term = f"{coeff}*{var}^{power}"
        poly_str += f"{sign}{term}"
    return poly_str.strip()


# ---- HÀM CHÍNH TẠO BÀI GIẢI THEO CẤU TRÚC MỚI ----

def baiGiaiMocCachDeuFinal(x_data, y_data, h):
    """
    Tái tạo lại bài giải chi tiết với Bảng Tích và gộp Bảng Thương.
    """
    n = len(x_data) - 1
    indices = np.arange(n + 1)

    print("\n================== BÀI GIẢI CHI TIẾT (Mốc cách đều) ==================\n")
    
    # 1. Vector Dy (y_i * D_i)
    print("1. Vector Dy (y_i * D_i):")
    D_y_list = []
    for i in range(n + 1):
        denominator = factorial(i) * ((-1)**(n-i)) * factorial(n-i)
        D_i = 1 / denominator
        D_y_list.append(y_data[i] * D_i)
    D_y_array = np.array(D_y_list)
    Dy_df = pd.DataFrame([D_y_array], columns=[f"Dy_{i}" for i in indices])
    print(Dy_df.to_string(index=False, float_format="{:,.8f}".format))

    # 2. Bảng Tích cho W_{n+1}(t)
    print("\n2. Bảng Tích xây dựng W_{n+1}(t) = t(t-1)...(t-n):")
    W_tich = np.array([1.0])
    tich_table_data = []
    max_len = n + 2
    for i in range(n + 1):
        W_tich = nhandathuc(W_tich, i)
        padding = [0.0] * (max_len - len(W_tich))
        row = [f"Nhân với (t-{i})"] + padding + list(W_tich)
        tich_table_data.append(row)
    
    tich_df = pd.DataFrame(tich_table_data)
    print(tich_df.to_string(header=False, index=False, float_format="{:,.0f}".format))
    print(f"   => W_{n+1}(t) = {format_poly_final(W_tich, var='t', precision=0)}")
    
    W_n1_t = W_tich # Lưu lại để dùng tiếp

    # 3. Bảng Thương và tính P(t)
    print("\n3. Bảng Thương và tính P(t):")
    W_i_matrix_data = []
    for i in range(n + 1):
        W_i_t = poly_divide_by_monomial(W_n1_t, i)
        W_i_matrix_data.append(W_i_t)
    
    P_t_coeffs = np.sum(D_y_array[:, None] * np.array(W_i_matrix_data), axis=0)
    
    # Chuẩn bị dữ liệu cho DataFrame
    # Thêm padding để tất cả các hàng có cùng độ dài với W_{n+1}(t)
    padded_W_i_data = [np.pad(row, (1, 0), 'constant') for row in W_i_matrix_data]
    padded_P_t_coeffs = np.pad(P_t_coeffs, (1, 0), 'constant')

    full_table_data = [W_n1_t] + padded_W_i_data + [padded_P_t_coeffs]
    
    table_index = [f'W_{n+1}(t)'] + [f"W_{i}(t)" for i in indices] + ['P(t)']
    table_columns = [f"t^{n+1-j}" for j in range(n + 2)]
    
    thuong_df = pd.DataFrame(full_table_data, index=table_index, columns=table_columns)
    print(thuong_df.to_string(float_format="{:,.5f}".format))

    # 4. Kết luận P(t)
    print(f"\n4. Kết luận đa thức theo biến t:")
    print(f"   P(t) = {format_poly_final(P_t_coeffs, var='t')}")
    
    return P_t_coeffs

# ---- HÀM ĐỔI BIẾN ----
def doiBienVaInKetQua(P_t_coeffs, x0, h):
    t, x = symbols('t x')
    P_t_expr = sum(coeff * t**(len(P_t_coeffs)-1-i) for i, coeff in enumerate(P_t_coeffs))
    P_x_expr = P_t_expr.subs(t, (x - x0) / h)
    P_x_expanded = expand(P_x_expr)
    print("\n================== KẾT QUẢ CUỐI CÙNG THEO BIẾN x ==================\n")
    print(f"Đổi biến ngược: t = (x - {x0})/{h}")
    print(f"P(x) = {N(P_x_expanded, 12)}")

# ---- HÀM TÍNH GIÁ TRỊ TẠI ĐIỂM CỤ THỂ ----
def tinhGiaTriTaiDiem(P_t_coeffs, x0, h, x_eval):
    def Tinh(coeffs, c): # Hàm Horner nội bộ
        b = coeffs[0]
        for i in range(1, len(coeffs)): b = b * c + coeffs[i]
        return b
        
    print("\n================== TÍNH GIÁ TRỊ TẠI ĐIỂM CỤ THỂ ==================\n")
    t_eval = (x_eval - x0) / h
    print(f"Giá trị cần tính: x = {x_eval}")
    print(f"Đổi sang biến t: t = ({x_eval} - {x0}) / {h} = {t_eval:.8f}")
    y_eval = Tinh(P_t_coeffs, t_eval)
    print(f"\nGiá trị của đa thức tại điểm này là:")
    print(f"P({x_eval}) = {y_eval:.8f}")

# ---- CHƯƠNG TRỊNH CHÍNH ----
if __name__ == "__main__":
    # Dữ liệu từ file Excel trong ảnh
    x_data, y_data = read_data_from_excel("input_lagrange_moc_cach_deu.xlsx")
    
    h = checkEquidistant(x_data)
    if h is not None:
        print(f"Phát hiện các mốc cách đều với bước nhảy h = {h:.4f}")
        x0 = x_data[0]
        
        # Chạy bài giải chi tiết theo cấu trúc mới
        P_t_coeffs = baiGiaiMocCachDeuFinal(x_data, y_data, h)
        
        # Đổi về biến x và in kết quả
        doiBienVaInKetQua(P_t_coeffs, x0, h)
        
        # Tính giá trị tại điểm yêu cầu
        x_value_to_calculate = 21.05931
        tinhGiaTriTaiDiem(P_t_coeffs, x0, h, x_value_to_calculate)
        
    else:
        print("Các mốc không cách đều.")

Phát hiện các mốc cách đều với bước nhảy h = 3.0000

================== BÀI GIẢI CHI TIẾT (Mốc cách đều) ==================

1. Vector Dy (y_i * D_i):
       Dy_0       Dy_1        Dy_2       Dy_3        Dy_4       Dy_5        Dy_6       Dy_7        Dy_8       Dy_9
-0.00000337 0.00008095 -0.00067718 0.00264120 -0.00580278 0.00728542 -0.00582037 0.00285764 -0.00078425 0.00008938

2. Bảng Tích xây dựng W_{n+1}(t) = t(t-1)...(t-n):
Nhân với (t-0) 0   0   0      0      0        0       0          0         0        1 -0
Nhân với (t-1) 0   0   0      0      0        0       0          0         1       -1  0
Nhân với (t-2) 0   0   0      0      0        0       0          1        -3        2 -0
Nhân với (t-3) 0   0   0      0      0        0       1         -6        11       -6  0
Nhân với (t-4) 0   0   0      0      0        1     -10         35       -50       24 -0
Nhân với (t-5) 0   0   0      0      1      -15      85       -225       274     -120  0
Nhân với (t-6) 0   0   0      1  